# ContrastiveExplanation
This file contains a Notebook for example usage of the `ContrastiveExplanation` Python script, so that you can familiarize yourself with the usage flow and showcasing the package functionalities. It contains two examples, one for explaining a classification problem and one for explaining a regression analysis problem.

First, let us set a seed for reproducibility, and import packages.

In [1]:
from numpy.random import RandomState
SEED = RandomState(1994)

In [2]:
from sklearn import datasets, model_selection, ensemble, metrics

For printing out the features and their corresponding values of an instance we define a function `print_sample()`:

In [3]:
import pprint

def print_sample(feature_names, sample):
    print('\n'.join(f'{name}: {value}' for name, value in zip(feature_names, sample)))

## Classification
First, for classification we use the [Iris](https://archive.ics.uci.edu/ml/datasets/iris) data set (as also used in the example of `README.md`. We first load the data set (and print its characteristics), then train an ML model on the data, and finally explain an instance predicted using the model.

---
**0. Data set characteristics**

In [4]:
data = datasets.load_iris()
print(data['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

**1. Train a (black-box) model on the Iris data**

In [5]:
# Split data in a train/test set and in predictor (x) and target (y) variables
x_train, x_test, y_train, y_test = model_selection.train_test_split(data.data, 
                                                                    data.target, 
                                                                    train_size=0.80, 
                                                                    random_state=SEED)

# Train a RandomForestClassifier
model = ensemble.RandomForestClassifier(random_state=SEED, n_estimators=100)
model.fit(x_train, y_train)

# Print out the classifier performance (F1-score)
print('Classifier performance (F1):', metrics.f1_score(y_test, model.predict(x_test), average='weighted'))

Classifier performance (F1): 0.9333333333333333


**2a. Perform contrastive explanation**

In [6]:
# Import
import contrastive_explanation as ce

# Select a sample to explain ('questioned data point') why it predicted the fact instead of the foil 
sample = x_test[1]
print_sample(data.feature_names, sample)

# Create a domain mapper (map the explanation to meaningful labels for explanation)
dm = ce.domain_mappers.DomainMapperTabular(x_train,
                                           feature_names=data.feature_names,
                                           contrast_names=data.target_names)

# Create the contrastive explanation object (default is a Foil Tree explanator)
exp = ce.ContrastiveExplanation(dm)

# Explain the instance (sample) for the given model
exp.explain_instance_domain(model.predict_proba, sample)

sepal length (cm): 5.1
sepal width (cm): 3.8
petal length (cm): 1.5
petal width (cm): 0.3


"The model predicted 'setosa' instead of 'versicolor' because 'petal length (cm) <= 2.528 and petal width (cm) <= 1.704 and sepal length (cm) <= 5.159'"

**2b. Perform contrastive explanation on manually selected foil**

Instead, we can also manually provide a foil to explain (e.g. class 'virginica'):

In [7]:
exp.explain_instance_domain(model.predict_proba, sample, foil='virginica')

"The model predicted 'setosa' instead of 'virginica' because 'petal length (cm) <= 5.133 and sepal length (cm) <= 6.059'"

## Regression
Here, we explain an instance of the [Diabetes](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes) data set using the same steps as the classification problem.

---
**0. Data set characteristics**

In [8]:
r_data = datasets.load_diabetes()
print(r_data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

**1. Train a (black-box) model on the Diabetes data**

In [9]:
# Split data in a train/test set and in predictor (x) and target (y) variables
rx_train, rx_test, ry_train, ry_test = model_selection.train_test_split(r_data.data, 
                                                                        r_data.target, 
                                                                        train_size=0.80, 
                                                                        random_state=SEED)

# Train a RandomForestRegressor with hyperparameter tuning (selecting the best n_estimators)
m_cv = ensemble.RandomForestRegressor(random_state=SEED)
r_model = model_selection.GridSearchCV(m_cv, cv=5, param_grid={'n_estimators': [50, 100, 500]})
r_model.fit(rx_train, ry_train)

# Print out the regressor performance
print('Regressor performance (R-squared):', metrics.r2_score(ry_test, r_model.predict(rx_test)))

Regressor performance (R-squared): 0.4054625517721193


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


**2. Perform contrastive explanation**

In [10]:
# Import
import contrastive_explanation as ce

# Select a sample to explain
r_sample = rx_test[1]
print_sample(r_data.feature_names, r_sample)
print('\n')

# Create a domain mapper (still tabular data, but for regression we do not have named labels for the outcome)
r_dm = ce.domain_mappers.DomainMapperTabular(rx_train, 
                                             feature_names=r_data.feature_names)

# Create the CE objects, ensure that 'regression' is set to True
# again, we use the Foil Tree explanator, but now we print out intermediary outcomes and steps (verbose)
r_exp = ce.ContrastiveExplanation(r_dm,
                                  regression=True,
                                  explanator=ce.explanators.TreeExplanator(verbose=True),
                                  verbose=False)

# Explain using the model, also include a 'factual' (non-contrastive 'why fact?') explanation
r_exp.explain_instance_domain(r_model.predict, r_sample, include_factual=True)

age: 0.0380759064334241
sex: 0.0506801187398187
bmi: -0.0299178197611881
bp: -0.0745280244296595
s1: -0.0125765826858204
s2: -0.0125872220506418
s3: 0.00446044580110504
s4: -0.00259226199818282
s5: 0.00371173823343597
s6: -0.0300724459043093


[E] Explaining with a decision tree...
[E] Fidelity of tree on neighborhood data = 1.0
[E] Found 15 contrastive decision regions, starting from node 2
[E] Found shortest path [42, 41, 43] using strategy "informativeness"


("The model predicted '128.38' instead of 'more than 128.38' because 's6 <= 0.069'",
 "The model predicted '128.38' because 'bmi <= 0.01 and s5 <= -0.026 and age > 0.021 and bmi > 0.001 and s4 <= -0.046 and s3 > -0.015 and sex > 0.085'")